In [1]:
%load_ext autoreload
%autoreload 2
PROJ_HOME='/Users/skeem396/Projects/analyst_performance/'
import sys
sys.path.append(PROJ_HOME)
import gc
from datetime import datetime

In [45]:
import pandas as pd
import copy
import pickle
from tqdm import tqdm

In [3]:
from pathlib import Path

In [4]:
data_dir = Path(PROJ_HOME+'/data')

# EDA

In [5]:
fnms = sorted(list(data_dir.glob('*summary.csv')))
data_lists = []
for i, fnm in enumerate(fnms):
    data_lists.append(pd.read_csv(fnm))
    print(fnm)
    print(data_lists[i].shape)

/Users/skeem396/Projects/nlp/nlp_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (0,1,2,3,4,5,6,7,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


/Users/skeem396/Projects/analyst_performance/data/2020_Part 1_summary.csv
(29999, 66)
/Users/skeem396/Projects/analyst_performance/data/2020_Part 2_summary.csv
(7000, 66)
/Users/skeem396/Projects/analyst_performance/data/2020_Part 3_summary.csv
(29999, 66)
/Users/skeem396/Projects/analyst_performance/data/2021_Part1_summary.csv
(29999, 66)
/Users/skeem396/Projects/analyst_performance/data/2021_Part2_summary.csv
(29999, 66)
/Users/skeem396/Projects/analyst_performance/data/2021_Part3_summary.csv
(29999, 66)


In [6]:
for d in data_lists:
    print(d.columns.to_list())

['작성자', '자료제공처', '종목코드', '종목명', '발행일', '발행 전일', '발행 전전일', '리포트 제목', '시가', '종가', '고가', '전일종가', '전전일종가', '투자의견', '고가1', '고가2', '고가3', '고가4', '고가5', '고가6', '고가7', '고가8', '고가9', '고가10', '고가11', '고가12', '고가13', '고가14', '고가15', '고가16', '고가17', '고가18', '고가19', '고가20', '고가21', '고가22', '고가23', '고가24', '고가25', '고가26', '고가27', '고가28', '고가29', '고가30', '고가31', '고가32', '고가33', '고가34', '고가35', '고가36', '고가37', '고가38', '고가39', '고가40', '고가41', '고가42', '고가43', '고가44', '고가45', '고가46', '고가47', '고가48', '고가49', '고가50', 'H25', 'H50']
['작성자', '자료제공처', '종목코드', '종목명', '발행일', '발행 전일', '발행 전전일', '리포트 제목', '시가', '종가', '고가', '전일종가', '전전일종가', '투자의견', '고가1', '고가2', '고가3', '고가4', '고가5', '고가6', '고가7', '고가8', '고가9', '고가10', '고가11', '고가12', '고가13', '고가14', '고가15', '고가16', '고가17', '고가18', '고가19', '고가20', '고가21', '고가22', '고가23', '고가24', '고가25', '고가26', '고가27', '고가28', '고가29', '고가30', '고가31', '고가32', '고가33', '고가34', '고가35', '고가36', '고가37', '고가38', '고가39', '고가40', '고가41', '고가42', '고가43', '고가44', '고가45', '고가46', '고가47', '고가48'

In [7]:
truncated_data = []
for i, dl in enumerate(data_lists):
    print(i)
    found = False
    for k, d in dl.iterrows():
        d.fillna(0, inplace=True)
        if d['작성자'] == 0:
            print(k)
            truncated_data.append(dl.iloc[:k])
            found = True
            break
    if not found:
        truncated_data.append(dl)

0
6999
1
2
9684
3
5115
4
5893
5
5201


In [8]:
for i, dl in enumerate(truncated_data):
    print(dl.shape)

(6999, 66)
(7000, 66)
(9684, 66)
(5115, 66)
(5893, 66)
(5201, 66)


In [9]:
type(data_lists[0].iloc[6999]['작성자'])

float

In [10]:
len(data_lists)

6

In [35]:
merged_df = pd.concat(truncated_data, axis=0)
print(merged_df.shape)
merged_df

(39892, 66)


,작성자,자료제공처,종목코드,종목명,발행일,발행 전일,발행 전전일,리포트 제목,시가,종가,...,고가43,고가44,고가45,고가46,고가47,고가48,고가49,고가50,H25,H50
0,백영찬,KB,A078930,GS,1/2/20,12/30/19,NaN,4분기실적은기대수준을소폭하회할전망,"51,300","50,000",...,41100.0,41300.0,41000.0,39400.0,40450.0,40450.0,40550.0,36400.0,"51,400","51,400"
1,백영찬,KB,A010060,OCI,1/2/20,12/30/19,NaN,폴리실리콘원가개선필요한시점,"62,700","62,800",...,50800.0,52200.0,52000.0,50100.0,48450.0,48200.0,46300.0,39150.0,"65,900","67,300"
2,백영찬,KB,A011780,금호석유,1/2/20,12/30/19,NaN,"견조한고무사업,그러나부진한페놀유도체","77,300","76,000",...,62200.0,66100.0,65900.0,64200.0,62800.0,63200.0,63700.0,56300.0,"82,100","82,100"
3,장정훈,삼성,A006400,삼성SDI,1/2/20,12/30/19,NaN,20년연초에꾸는꿈,"237,000","232,000",...,318000.0,324500.0,323500.0,305500.0,298000.0,301500.0,291500.0,284500.0,"322,500","351,000"
4,백영찬,KB,A051910,LG화학,1/2/20,12/30/19,NaN,4분기ESS비용반영을긍정적으로보자,"321,000","314,000",...,398500.0,412000.0,404000.0,385500.0,380000.0,377500.0,367000.0,343500.0,"391,500","422,500"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5196,김록호,하나금융,A009150,삼성전기,9/10/21,9/9/21,9/8/21,호실적과 편안한 2022년,"182,500","188,000",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"188,000","188,000"
5197,최정욱,하나금융,A316140,우리금융지주,9/10/21,9/9/21,9/8/21,공자위의 조속한 매각 결단을 환영,"10,850","10,950",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"10,950","10,950"
5198,심은주,하나금융,A271560,오리온,9/10/21,9/9/21,9/8/21,저가 매수 유효,"124,500","123,500",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"124,500","124,500"
5199,박강호,대신,A229640,LS전선아시아,9/10/21,9/9/21,9/8/21,"숨은 진주, 3Q 호실적 & 신재생에너지","8,840","8,770",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"8,880","8,880"


In [36]:
merged_df.rename(columns={'작성자':'author', '자료제공처': 'source', '종목코드': 'ticker_symbol', '종목명': 'company', '발행일': 'report_date', '리포트 제목': 'report_title', '시가': 'opening', '종가': 'closing', '투자의견': 'recommendation'}, inplace=True)

In [37]:
merged_df['author'].value_counts()

김경민       272
김현용       255
   박종선    230
백영찬       210
   이동헌    204
         ... 
임규미         1
   김규상      1
최길환         1
안도영         1
   송용주      1
Name: author, Length: 865, dtype: int64

In [33]:
col_kor_names = merged_df.columns.to_list()

In [34]:
col_name_map = col_kor_names

In [38]:
merged_df['recommendation'].value_counts()

없음               14870
BUY              13079
NR                6582
매수                4113
HOLD               700
중립                 189
MarketPerform      126
Trading Buy         63
Neutral             51
Outperform          45
STRONG BUY          38
보유                  22
비중축소                 4
Underperform         4
Sell                 3
REDUCE               3
Name: recommendation, dtype: int64

In [19]:
import spacy

In [20]:
nlp = spacy.load("en_core_web_lg")

In [17]:
from googletrans import Translator

In [18]:
translator = Translator(service_urls=[
      'translate.google.com',
      'translate.google.co.kr',
    ])

In [22]:
ex1 = translator.translate('안녕하세요.', src="ko", dest="en")

In [27]:
print(ex1)

Translated(src=ko, dest=en, text=Hello., pronunciation=None, extra_data="{'confiden...")


In [28]:
ex1.text

'Hello.'

In [30]:
ex1.extra_data

{'confidence': None,
 'parts': [<googletrans.models.TranslatedPart at 0x7fde8d360ad0>],
 'origin_pronunciation': 'annyeonghaseyo.',
 'parsed': [['annyeonghaseyo.', None, None, [[[0, [[[None, 6]], [True]]]], 6]],
  [[[None,
     None,
     None,
     None,
     None,
     [['Hello.',
       None,
       None,
       None,
       [['Hello.', [4, 5]], ['Good morning.', [2, 1]], ['Hi.', [2]]]]]]],
   'en',
   1,
   'ko',
   ['안녕하세요.', 'ko', 'en', True]],
  'ko',
  ['안녕하세요!',
   None,
   None,
   None,
   None,
   [[['interjection',
      [['Hello!',
        None,
        ['안녕하세요!', '여보!', '어이!', '여보, 이 봐!', '여보세요!'],
        1,
        True],
       ['Good morning!', None, ['안녕하세요!', '안녕하십니까!', '안녕히 주무셨어요!'], 2, True],
       ['Good afternoon!', None, ['안녕하세요!'], 3, True],
       ['Good day!', None, ['안녕하세요!'], 3, True],
       ['Good evening!', None, ['안녕하세요!'], 3, True]],
      'en',
      'ko']],
    5],
   None,
   None,
   'ko',
   1]]}

In [48]:
merged_df.to_pickle(PROJ_HOME+'/data/merged.pkl')

In [ ]:
eng_title = []
for k, r in tqdm(merged_df.iterrows(), total=39892):
    ex1 = translator.translate(r['report_title'] , src="ko", dest="en")
    eng_title.append(ex1.text)

In [ ]:
merged_df['en_report_title'] = eng_title

In [ ]:
merged_df

In [ ]:
merged_df.to_pickle(PROJ_HOME+'/data/merged.pkl')